In [1]:
import keras

from keras.models import Model
from keras.models import Sequential
from keras.models import load_model

from keras.layers import Activation
from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM,SimpleRNN

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical

import jieba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# 导入自定义库
from utils.data_utils import clean_str
from utils.data_utils import build_vocab
from utils.data_utils import get_tokens

In [3]:
# 加载训练集合
df_train_dataset = pd.read_csv('./data/training-inspur.csv', encoding='utf-8')
# 加载测试集
df_test_dataset = pd.read_csv('./data/Preliminary-texting-1.csv', encoding='utf-8')
# 查看数据集合 shape
df_test_dataset.shape

(102024, 2)

In [4]:
# 提取数据集中所需的字段
df_train_dataset = df_train_dataset[['COMMCONTENT', 'COMMLEVEL']]
df_test_dataset = df_test_dataset[['COMMCONTENT']]

In [5]:
# 合并数据集用于构建词汇表
df_all_dataset = pd.concat([df_train_dataset, df_test_dataset], ignore_index=True)
df_all_dataset.shape

(122024, 2)

In [6]:
BASE_DIR = '/Users/tsw/ScenicSpotReviews'

W2V_DIR = BASE_DIR + '/embeddings/'

TEXT_DATA_DIR = BASE_DIR + '/data/'

MAX_NUM_WORDS = 33950

MAX_SEQUENCE_LENGTH = 150 # 每篇文章选取150个词

MAX_NB_WORDS = 80000 # 将字典设置为含有1万个词84480

EMBEDDING_DIM = 300 # 词向量维度，300维

VALIDATION_SPLIT = 0.1 # 测试集大小，全部数据的10%

BATCH_SIZE = 128

NUM_LABELS = 3

In [7]:
def seg_corpus(corpus):
    seg_corpus = []
    for line in corpus:
        line = str(line).strip()
        seg_list = jieba.cut(line, cut_all=False)
        # 过滤空字符
        seg_list = [w for w in seg_list if w != ' ']
        seg_corpus.append(" ".join(seg_list))
    return seg_corpus

In [8]:
# 对所有文本分词
seged_text = seg_corpus(df_all_dataset['COMMCONTENT'])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/5n/2_by50851fxc4d_snc1d9wf80000gn/T/jieba.cache
Loading model cost 3.185 seconds.
Prefix dict has been built succesfully.


In [9]:
# 将分词后的数据并入 df_all_dataset
df_all_dataset['COMMCONTENT_SEG'] = pd.DataFrame(seged_text,columns=['COMMCONTENT_SEG'])
df_all_dataset.head()

,COMMCONTENT,COMMLEVEL,COMMCONTENT_SEG
0,普通公园一个只是多了几个泉而已，人不多，适合老人孩子闲逛，买票的话还是贵了，人家说6.30之...,1.0,普通 公园 一个 只是 多 了 几个 泉 而已 ， 人不多 ， 适合 老人 孩子 闲逛 ， ...
1,跟儿子在里面玩了一天，非常好！跟儿子在里面玩了一天，非常好！真的很不错哦，有空还要去,1.0,跟 儿子 在 里面 玩 了 一天 ， 非常 好 ！ 跟 儿子 在 里面 玩 了 一天 ， 非...
2,这已经是第五次来这里玩了。每次孩子都很喜欢，不愿意从水里出来。有机会还会再来。还有比我更忠诚...,1.0,这 已经 是 第五次 来 这里 玩 了 。 每次 孩子 都 很 喜欢 ， 不 愿意 从水里 ...
3,当天在携程上定的票，打温泉度假村咨询电话和携程客服都说次日生效，但到酒店后，票能用。请客服人...,1.0,当天 在 携程 上定 的 票 ， 打 温泉 度假村 咨询电话 和 携程 客服 都 说 次日 ...
4,烟台历史的一部分，非常值得推荐去看看！海边景色也很漂亮！,1.0,烟台 历史 的 一部分 ， 非常 值得 推荐 去 看看 ！ 海边 景色 也 很漂亮 ！


In [10]:
# text_corpus
text_corpus = df_all_dataset['COMMCONTENT_SEG']
# 传入我们词向量的字典
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) 
# 传入我们的训练数据，得到训练数据中出现的词的字典
tokenizer.fit_on_texts(text_corpus) 

In [11]:
# 根据训练数据中出现的词的字典，将训练数据转换为sequences
dataset_sequences = tokenizer.texts_to_sequences(text_corpus) 

In [12]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 100134 unique tokens.


In [13]:
padded_dataset_sequences = pad_sequences(dataset_sequences, maxlen=MAX_SEQUENCE_LENGTH)
padded_dataset_sequences.shape

(122024, 150)

In [27]:
df_test_dataset_seg = df_all_dataset['COMMCONTENT_SEG'][20000:]
test_dataset_sequences = tokenizer.texts_to_sequences(df_test_dataset_seg)
padded_test_dataset_sequences = pad_sequences(test_dataset_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [14]:
# 划分训练集和测试集
from sklearn.model_selection import train_test_split
train_X,valid_X,train_y,valid_y =train_test_split(padded_dataset_sequences[:df_train_dataset.shape[0]], 
                                                  df_all_dataset['COMMLEVEL'][:df_train_dataset.shape[0]], 
                                                  test_size=0.1)

In [15]:
# label one-hot 表示
labels = df_all_dataset['COMMLEVEL'].dropna().map(int)#.values.tolist()
labels = to_categorical(labels-1) 



In [16]:
vocab,vocab_freqs = build_vocab(df_all_dataset['COMMCONTENT_SEG'])
vocab_size = min(MAX_NB_WORDS, len(vocab_freqs)) + 2
word2index = {x[0]: i+2 for i, x in enumerate(vocab_freqs.most_common(MAX_NB_WORDS))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}
len(word2index)

80002

In [17]:
print('Indexing word embeddings.')  
embeddings_index = {}
with open('./embeddings/sgns.weibo.word','r') as f:
    f = f.readlines()
    for i in f[1:]:
        values = i.strip().split(' ')
        word = str(values[0])
        embedding = np.asarray(values[1:],dtype='float')
        embeddings_index[word] = embedding
print('word embedding',len(embeddings_index))

Indexing word embeddings.
word embedding 195201


In [18]:
nb_words = min(MAX_NB_WORDS,len(word2index))
nb_words

80000

In [19]:
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

In [20]:
for word, i in word2index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(str(word).upper())
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

In [38]:
pre_embedding_layer =  Embedding(input_dim = nb_words+1, 
                             output_dim = EMBEDDING_DIM, 
                            weights=[word_embedding_matrix], 
                             input_length=MAX_SEQUENCE_LENGTH, 
                             mask_zero=True,
                             trainable=False
                            )

In [47]:
from keras.layers import Lambda
from keras import backend as K

In [62]:
def get_rnn_cnn_model():
    embedding_dim = 300
    inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    x =  Embedding(input_dim = nb_words+1, 
                             output_dim = EMBEDDING_DIM, 
                             weights=[word_embedding_matrix], 
                             input_length=MAX_SEQUENCE_LENGTH, 
                             mask_zero=False,
                             trainable=True
                            )(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    kmax_pool = Lambda(lambda x: K.max(x, axis=1), output_shape=(64,))(x)
    conc = concatenate([avg_pool, max_pool, kmax_pool])
    outp = Dense(3, activation="softmax")(conc)

    model = Model(inputs=inp, outputs=outp)
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [63]:
rnn_cnn_model = get_rnn_cnn_model()

plot_model(rnn_cnn_model, to_file='./rnn_cnn_model.png', 
show_shapes=True, show_layer_names=True)

!['./rnn_cnn_model.png'](./rnn_cnn_model.png)

In [36]:
batch_size = 256
epochs = 4

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/4
18000/18000 [==============================] - 111s 6ms/step - loss: 0.6449 - acc: 0.7257 - val_loss: 0.7952 - val_acc: 0.6540
Epoch 2/4
18000/18000 [==============================] - 167s 9ms/step - loss: 0.6247 - acc: 0.7363 - val_loss: 0.7994 - val_acc: 0.6590
Epoch 3/4
18000/18000 [==============================] - 162s 9ms/step - loss: 0.6102 - acc: 0.7383 - val_loss: 0.7966 - val_acc: 0.6550
Epoch 4/4
18000/18000 [==============================] - 136s 8ms/step - loss: 0.5824 - acc: 0.7567 - val_loss: 0.8132 - val_acc: 0.6460


In [37]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=1024)
w2v7 = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(w2v7).value_counts(normalize=True)

2    0.386429
1    0.352074
3    0.261497
dtype: float64

In [41]:
# tainable = True
batch_size = 256
epochs = 4

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/4
18000/18000 [==============================] - 161s 9ms/step - loss: 0.9939 - acc: 0.4934 - val_loss: 0.8558 - val_acc: 0.6075
Epoch 2/4
18000/18000 [==============================] - 151s 8ms/step - loss: 0.7919 - acc: 0.6446 - val_loss: 0.7624 - val_acc: 0.6525
Epoch 3/4
18000/18000 [==============================] - 151s 8ms/step - loss: 0.6897 - acc: 0.6982 - val_loss: 0.7643 - val_acc: 0.6560
Epoch 4/4
18000/18000 [==============================] - 167s 9ms/step - loss: 0.6004 - acc: 0.7483 - val_loss: 0.7472 - val_acc: 0.6675


In [ ]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=1024)
w2v8 = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(w2v8).value_counts(normalize=True)

In [55]:
# add kmax pooling

In [56]:
batch_size = 256
epochs = 4

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/4
18000/18000 [==============================] - 165s 9ms/step - loss: 0.9959 - acc: 0.4907 - val_loss: 0.8695 - val_acc: 0.5880
Epoch 2/4
18000/18000 [==============================] - 159s 9ms/step - loss: 0.7919 - acc: 0.6396 - val_loss: 0.7781 - val_acc: 0.6475
Epoch 3/4
18000/18000 [==============================] - 162s 9ms/step - loss: 0.6880 - acc: 0.7008 - val_loss: 0.7498 - val_acc: 0.6695
Epoch 4/4
18000/18000 [==============================] - 156s 9ms/step - loss: 0.5987 - acc: 0.7523 - val_loss: 0.7466 - val_acc: 0.6755


In [58]:
batch_size = 256
epochs = 1

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 158s 9ms/step - loss: 0.4216 - acc: 0.8386 - val_loss: 0.8321 - val_acc: 0.6685


In [59]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=256)
rnn_cnn_k_max = np.argmax(all_test_preds,axis=1)[:]+1

In [60]:
pd.Series(rnn_cnn_k_max).value_counts(normalize=True)

1    0.397642
3    0.361670
2    0.240688
dtype: float64

In [61]:
np.savetxt("rnn-cnn-0.6685-result.txt", rnn_cnn_k_max,fmt="%d")

In [64]:
batch_size = 256
epochs = 5

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/5
18000/18000 [==============================] - 160s 9ms/step - loss: 1.0040 - acc: 0.4832 - val_loss: 0.8355 - val_acc: 0.6260
Epoch 2/5
18000/18000 [==============================] - 156s 9ms/step - loss: 0.7918 - acc: 0.6458 - val_loss: 0.7754 - val_acc: 0.6505
Epoch 3/5
18000/18000 [==============================] - 156s 9ms/step - loss: 0.6891 - acc: 0.7009 - val_loss: 0.7527 - val_acc: 0.6635
Epoch 4/5
18000/18000 [==============================] - 152s 8ms/step - loss: 0.5947 - acc: 0.7536 - val_loss: 0.7569 - val_acc: 0.6740
Epoch 5/5
18000/18000 [==============================] - 154s 9ms/step - loss: 0.5116 - acc: 0.7956 - val_loss: 0.7846 - val_acc: 0.6770


In [65]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=256)
rnn_cnn_k_max2 = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(rnn_cnn_k_max2).value_counts(normalize=True)

1    0.377529
3    0.369825
2    0.252646
dtype: float64

In [66]:
np.savetxt("rnn-cnn-0.6770-result.txt", rnn_cnn_k_max2,fmt="%d")

In [29]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=256)
w2v6 = np.argmax(all_test_preds,axis=1)[:]+1

In [30]:
pd.Series(w2v6).value_counts(normalize=True)

1    0.423273
3    0.330932
2    0.245795
dtype: float64

In [32]:
np.savetxt("rnn-cnn-0.6320-result.txt", w2v6,fmt="%d")

In [ ]:
print('Build model...')
model = Sequential()
model.add(pre_embedding_layer)
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1, return_sequences=True)))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1)))
model.add(Dense(NUM_LABELS, activation='softmax'))

In [72]:
def get_rnn_cnn_model():
    embedding_dim = 300
    inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    x =  Embedding(input_dim = nb_words+1, 
                             output_dim = EMBEDDING_DIM, 
                             weights=[word_embedding_matrix], 
                             input_length=MAX_SEQUENCE_LENGTH, 
                             mask_zero=False,
                             trainable=False
                            )(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
#     max_pool = GlobalMaxPooling1D()(x)
    kmax_pool = Lambda(lambda x: K.max(x, axis=1), output_shape=(64,))(x)
    conc = concatenate([avg_pool, kmax_pool])
    outp = Dense(3, activation="softmax")(conc)

    model = Model(inputs=inp, outputs=outp)
    
    model.compile(loss='categorical_crossentropy',
              #optimizer='adam',
                  optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])
    
    return model

In [73]:
rnn_cnn_model = get_rnn_cnn_model()

In [71]:
batch_size = 256
epochs = 5

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/5
18000/18000 [==============================] - 170s 9ms/step - loss: 1.0061 - acc: 0.4828 - val_loss: 0.8898 - val_acc: 0.5995
Epoch 2/5
18000/18000 [==============================] - 154s 9ms/step - loss: 0.8893 - acc: 0.5828 - val_loss: 0.8448 - val_acc: 0.6200
Epoch 3/5
18000/18000 [==============================] - 159s 9ms/step - loss: 0.8430 - acc: 0.6107 - val_loss: 0.8212 - val_acc: 0.6430
Epoch 4/5
18000/18000 [==============================] - 174s 10ms/step - loss: 0.8132 - acc: 0.6263 - val_loss: 0.8134 - val_acc: 0.6410
Epoch 5/5
18000/18000 [==============================] - 153s 9ms/step - loss: 0.7819 - acc: 0.6469 - val_loss: 0.8271 - val_acc: 0.6380


In [74]:
batch_size = 256
epochs = 5

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/5
18000/18000 [==============================] - 155s 9ms/step - loss: 1.0383 - acc: 0.4613 - val_loss: 0.9236 - val_acc: 0.5565
Epoch 2/5
18000/18000 [==============================] - 160s 9ms/step - loss: 0.9044 - acc: 0.5735 - val_loss: 0.8621 - val_acc: 0.6180
Epoch 3/5
18000/18000 [==============================] - 154s 9ms/step - loss: 0.8567 - acc: 0.5992 - val_loss: 0.8289 - val_acc: 0.6245
Epoch 4/5
  768/18000 [>.............................] - ETA: 2:27 - loss: 0.7897 - acc: 0.6510

KeyboardInterrupt: 